<a href="https://colab.research.google.com/github/Admindatosgobes/Laboratorio-de-Datos/blob/main/Data%20Science/Asistente%20Experto%20sobre%20Minerales%20Cr%C3%ADticos/C%C3%B3digo/pdf_a_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PDF a CSV

Script Python para la extracción de datos del PDF de JRC desarrollado en plataforma Google Colab.

In [ ]:
# Importamos las librerías necesarias para la programación del script
import PyPDF2
import pandas as pd

In [ ]:
# Abrimos el fichero PDF
ficheroPdf = open('JRC_Supply chain analysis and material demand_2023.pdf', 'rb')

# Creamos el lector para poder acceder a su contenido
lectorPDF = PyPDF2.PdfReader(ficheroPdf)

# Extraemos las 3 páginas que contienen la tabla de datos que queremos analizar
# La variable "paginas" contendrá un array donde cada elemento será una fila de
# la tabla
paginas = []
paginas.append(lectorPDF.pages[240].extract_text().split("Risk Material")[1].split("\n")[1:-1])
paginas.append(lectorPDF.pages[241].extract_text().split("Risk Material")[1].split("\n")[1:-1])
paginas.append(lectorPDF.pages[242].extract_text().split("Risk Material")[1].split("\n")[1:-3])
paginas = paginas[0]+paginas[1]+paginas[2]

In [ ]:
# Recorremos línea a línea insertando en un array bi-dimensional (tabla) los valores del PDF
tabla = []
for i in range(len(paginas)):
    linea_texto = paginas[i][:]
    fila_tabla = linea_texto.split("  ",1)
    fila_tabla[0] = fila_tabla[0].split(maxsplit=1)
    fila_tabla[0][0] = float(fila_tabla[0][0])
    fila_tabla[1] = fila_tabla[1].replace("• ", "1")
    fila_tabla[1] = fila_tabla[1].replace(" ","0")
    if(len(fila_tabla[1])<15):
        fila_tabla[1]="0"+fila_tabla[1]
    fila_tabla[1] = [caracter for caracter in fila_tabla[1]]

    tabla.append(fila_tabla[0]+fila_tabla[1])

# Cargamos los valores en una estructura de datos para poder analizarlos
columnas = ['Riesgo Suministro','Nombre Material','Baterias Li-ion','Pilas de Combustible','Electrolizadores','Aerogeneradores','Motores de Traccion',
            'Solar Fotovoltaica','Bombas de Calor','Hidrogeno DRI','Redes de Transmision de Datos','Almacenamiento de Datos y Servidores',
            'Telefonos Inteligentes, Tabletas y Computadoras Portatiles','Fabricacion Aditiva','Robotica','Drones','Lanzadores Espaciales y Satelites']
datos = pd.DataFrame(tabla, columns=columnas)
datos.head()

,Riesgo Suministro,Nombre Material,Baterias Li-ion,Pilas de Combustible,Electrolizadores,Aerogeneradores,Motores de Traccion,Solar Fotovoltaica,Bombas de Calor,Hidrogeno DRI,Redes de Transmision de Datos,Almacenamiento de Datos y Servidores,"Telefonos Inteligentes, Tabletas y Computadoras Portatiles",Fabricacion Aditiva,Robotica,Drones,Lanzadores Espaciales y Satelites
0,5.6,Dysprosium,0,0,0,1,1,0,1,0,0,1,1,0,1,1,1
1,4.9,Terbium,0,0,0,1,0,0,0,0,0,1,0,0,1,1,1
2,4.8,Gallium,0,0,0,0,0,1,0,0,1,1,1,0,1,1,1
3,4.1,Magnesium,0,0,1,0,0,0,0,0,0,1,1,1,1,1,1
4,4.0,Cerium,0,1,1,0,0,0,0,0,0,0,1,0,1,1,1


In [ ]:
# Añadimos la columna que clasifica los materiales por Estratégicos, Críticos y No Críticos
datos.drop(columns=['Tipo Material'],errors="ignore", inplace=True)
datos.insert(loc=2, column="Tipo Material", value="Estrategico")
datos.loc[26:, 'Tipo Material'] = 'Critico' # A partir de Erbium
datos.loc[51:, 'Tipo Material'] = 'No critico' # A partir de Natural Teak wood

In [ ]:
# Exportamos el conjunto de datos en un fichero CSV
datos.to_csv('jrc_materiales_tecnologias.csv', index=False, decimal=',')